In [3]:

from SBMLReduce import *
sr = SBMLReduce()

n = 4 # Number of states 
nouts = 1 # Number of outputs
# Either
# sr.load_SBML_model('my_sbml_model.xml')

# OR write ODEs
#      x = E, S, ES, P
#      P = a, d, k, Etot

# parameter values
P = np.zeros(4)
P[0] = 10
P[1] = 10
P[2] = 0.1
P[3] = 1
sr.params_values = P.copy()
sr.timepoints = np.linspace(0, 2000, 100) # timepoints for simulation

x_init = np.zeros(n) # Initial conditions
x_init[0] = 1
x_init[1] = 0.5
sr.x_init = x_init

sr.error_tol = 100
sr.nstates_tol = 1
x, f, P = sr.load_ODE_model(n, len(sr.params_values))
params = P
# T1 and A1
f[0] = -P[0]*(P[3] - x[2])*x[1] + P[1]*x[2] + P[2]*x[2]
f[1] = -P[0]*(P[3] - x[2])*x[1] + P[1]*x[2]
f[2] = P[0]*(P[3] - x[2])*x[1] - (P[1] + P[2])*x[2]
f[3] = P[2]*x[2]

C = np.zeros((nouts,len(x)), dtype=int)
C[0][3] = 1
C = C.tolist()
sr.C = C

# f_hat_SBMLDocument = sr.get_reduced_model('SBML')
# f_hat_Scipy_callable_object = sr.get_reduced_model('ODE')


In [7]:
C

[[0, 0, 0, 1]]

In [5]:
sr.compute_reduced_model()

C:\Users\apand\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\integrate\_ivp\common.py:41: UserWarning: The following arguments have no effect for a chosen solver: `jac`.
  .format(", ".join("`{}`".format(x) for x in extraneous)))


Solving sensitivity for parameter 1/4       
Solving sensitivity for parameter 2/4       
Solving sensitivity for parameter 3/4       
Solving sensitivity for parameter 4/4       


C:\Users\apand\synBio\sbmlReduce\SBMLReduce.py:287: RuntimeWarning: invalid value encountered in double_scalars
  normed_sens[j,k] = normed_sens[j,k] * params_values[i] / x_sols[k,j]


IndexError: list index out of range

In [ ]:
f_hat = sr.get_reduced_model('Sympy')
print(f_hat)

In [2]:
from scipy.integrate import solve_ivp 
import numpy as np
timepoints = np.linspace(0,10,100)
def test(t,x):
    return -x[0] - x[1]
sol = solve_ivp(test, [0,0], timepoints)